In [2]:
import pandas as pd

# Carga el dataset enriquecido
final_path = "../../data/ridership_enriched_final.parquet"
df = pd.read_parquet(final_path)

print(f"Número de filas antes de agregar: {len(df)}")

# 1. Crea una columna de timestamp redondeada a la hora para agrupar
#    SOLUCIÓN 1: Cambiamos 'H' por 'h' minúscula.
df['hour_timestamp'] = df['transit_timestamp'].dt.floor('h')

# 2. Agrupa por ruta y por la nueva columna de hora
#    SOLUCIÓN 2: Añadimos explícitamente `observed=False`.
df_agg = df.groupby(['bus_route', 'hour_timestamp'], observed=False).agg(
    ridership_total=('ridership', 'sum'),
    transfers_total=('transfers', 'sum'),
    operating_buses=('bus_route', 'count')
).reset_index()

# 3. Une las variables enriquecidas
enriched_cols = [
    'hour', 'day_of_week', 'month', 'year', 'is_weekend', 'is_holiday',
    'temperature_2m', 'relative_humidity_2m', 'precipitation', 'rain',
    'snowfall', 'weather_code', 'wind_speed_10m'
]
df_enriched_unique = df.drop_duplicates(subset=['bus_route', 'hour_timestamp'])
df_model_input = pd.merge(
    df_agg,
    df_enriched_unique[['bus_route', 'hour_timestamp'] + enriched_cols],
    on=['bus_route', 'hour_timestamp']
)

# 4. Ordenamos el resultado final
df_model_input.sort_values(by=['bus_route', 'hour_timestamp'], inplace=True)

print(f"\nNúmero de filas después de agregar: {len(df_model_input)}")
print("\nDataFrame final listo para el modelo:")
print(df_model_input.head())

# 5. Guarda este DataFrame final
model_input_path = "../../data/model_input.parquet"
df_model_input.to_parquet(model_input_path, index=False)
print(f"\nDataset para el modelo guardado en: {model_input_path}")

Número de filas antes de agregar: 630720

Número de filas después de agregar: 26280

DataFrame final listo para el modelo:
  bus_route      hour_timestamp  ridership_total  transfers_total  \
0       M15 2023-01-01 00:00:00              323               26   
1       M15 2023-01-01 01:00:00              300               22   
2       M15 2023-01-01 02:00:00              144               19   
3       M15 2023-01-01 03:00:00              101               18   
4       M15 2023-01-01 04:00:00               38                0   

   operating_buses  hour  day_of_week  month  year  is_weekend  is_holiday  \
0               24     0            6      1  2023        True        True   
1               24     1            6      1  2023        True        True   
2               24     2            6      1  2023        True        True   
3               24     3            6      1  2023        True        True   
4               24     4            6      1  2023        True        Tr